## Building Hamiltonians

QMProt directly provides the Hamiltonians of the molecules included in the dataset. Nevertheless, we also provide the code used to compute them, both for replication purposes and to enable the computation of Hamiltonians for other molecules not included in the dataset, by modifying the atomic coordinates and elements of the system.

In [ ]:
!pip install pennylane quimb==1.8.2 > h

In [2]:
!pip install pyscf > h

In [3]:
!pip install basis-set-exchange > h


In [4]:
!pip install openfermionpyscf > h

In [ ]:
from openfermion import MolecularData, jordan_wigner
from openfermionpyscf import run_pyscf
import pennylane as qml
import pickle
from pennylane import qchem
from pennylane import numpy as np

def calculate_total_electrons(symbols):
    """Calculate the total number of electrons in the molecule."""
    electron_counts = {
        "H": 1, "C": 6, "N": 7, "O": 8, "S": 16  
    }
    return sum(electron_counts[symbol] for symbol in symbols)

#aspartic acid
coordinates = [
    [0.8765, -1.7477, 0.4612],   # O
    [-2.5468, 0.2085, -1.0110],  # O
    [2.0079, -0.5891, -1.1272],  # O
    [-1.8021, -0.7792, 0.8987],  # O
    [1.7445, 1.6788, 0.4055],    # N
    [0.7272, 0.6375, 0.5499],    # C
    [-0.5957, 1.1159, -0.0613],  # C
    [1.2748, -0.5980, -0.1470],  # C
    [-1.6864, 0.0732, 0.0312],   # C
    [0.6051, 0.4158, 1.6166],    # H
    [-0.4391, 1.3530, -1.1219],  # H
    [-0.9553, 2.0203, 0.4426],   # H
    [2.5932, 1.3992, 0.8965],    # H
    [1.4247, 2.5341, 0.8586],    # H
    [1.2403, -2.5417, 0.0146],   # H
    [-3.2665, -0.4573, -0.9797]  # H
]

symbols = [
    "O", "O", "O", "O", "N", "C", "C", "C", "C", "H", "H", "H", "H", "H", "H", "H"
]

def detect_multiplicity(symbols):
    total_electrons = calculate_total_electrons(symbols)
    if total_electrons % 2 == 0:
        return 1  
    else:
        return 2  

# Convert coordinates to the required format for OpenFermion
geometry = [(symbols[i], coordinates[i]) for i in range(len(symbols))]

total_electrons = calculate_total_electrons(symbols)
print(f"Total electrons in the molecule: {total_electrons}")

charge = 0  
multiplicity = detect_multiplicity(symbols) 

if (total_electrons + charge) % 2 != 0:
    raise ValueError("Inconsistent number of electrons and charge: Open-shell system detected.")

print("Charge and multiplicity are consistent with the number of electrons.")

basis = "sto-3g"

molecule = MolecularData(geometry=geometry, basis=basis, multiplicity=multiplicity, charge=charge)

molecule = run_pyscf(molecule, run_scf=True, run_fci=False)

fermionic_hamiltonian = molecule.get_molecular_hamiltonian()

qubit_hamiltonian = jordan_wigner(fermionic_hamiltonian)

output_file = "C4H7NO4_hamiltonian.txt"

with open(output_file, "w") as f:
    f.write("Coefficient\tOperators\n") 

    for term, coeff in qubit_hamiltonian.terms.items():
        
        ops = []
        num_qubits = max([q[0] for q in term], default=-1) + 1 
        for qubit in range(num_qubits):
            pauli_op = next((q[1] for q in term if q[0] == qubit), "I")  
            if pauli_op == "X":
                ops.append(qml.PauliX(qubit))
            elif pauli_op == "Y":
                ops.append(qml.PauliY(qubit))
            elif pauli_op == "Z":
                ops.append(qml.PauliZ(qubit))
     
        if ops:
            operator_string = " @ ".join([str(op) for op in ops])  
        else:
            operator_string = "Identity(0)"  
        f.write(f"{coeff}\t{operator_string}\n")

print(f"todo bien guardado.")


Total electrons in the molecule: 40
Charge and multiplicity are consistent with the number of electrons.
todo bien guardado.


In [4]:
num_qubits

60

In [5]:
len(qubit_hamiltonian.terms)

1164627